In [2]:
from transformers import Gemma2ForSequenceClassification, BitsAndBytesConfig, AutoTokenizer
import torch
from peft import PeftModel

/home/jovyan/.mlspace/envs/311venv_kotov/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cfg = {
    "_load_in_4bit": False,
    "_load_in_8bit": True,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": False,
    "llm_int8_enable_fp32_cpu_offload": False,
    "llm_int8_has_fp16_weight": False,
    "llm_int8_skip_modules": None,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": False,
    "load_in_8bit": True,
    "quant_method": "bitsandbytes"
  }

bnb_cfg = BitsAndBytesConfig(**cfg)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [9]:
model = Gemma2ForSequenceClassification.from_pretrained(
    'unsloth/gemma-2-9b-it',
    #quantization_config=bnb_cfg,
    num_labels=3,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:57<00:00, 14.39s/it]
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = PeftModel.from_pretrained(model, 'lmsys-model')

In [13]:
model = model.merge_and_unload()

In [14]:
model.score = torch.nn.Linear(in_features=3584, out_features=2, bias=False)

In [15]:
tokenizer = AutoTokenizer.from_pretrained('lmsys-model')

In [16]:
model.save_pretrained('MAIN-8BIT-LMSYS-PRETRAIN')
tokenizer.save_pretrained('MAIN-8BIT-LMSYS-PRETRAIN')

('MAIN-8BIT-LMSYS-PRETRAIN/tokenizer_config.json',
 'MAIN-8BIT-LMSYS-PRETRAIN/special_tokens_map.json',
 'MAIN-8BIT-LMSYS-PRETRAIN/tokenizer.model',
 'MAIN-8BIT-LMSYS-PRETRAIN/added_tokens.json',
 'MAIN-8BIT-LMSYS-PRETRAIN/tokenizer.json')

In [4]:
model = Gemma2ForSequenceClassification.from_pretrained(
    'MAIN-8BIT-LMSYS-PRETRAIN',
    quantization_config=bnb_cfg,
    num_labels=2,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.38s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained('MAIN-8BIT-LMSYS-PRETRAIN')

In [6]:
model.save_pretrained('QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN')
tokenizer.save_pretrained('QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN')

('QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN/tokenizer_config.json',
 'QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN/special_tokens_map.json',
 'QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN/tokenizer.model',
 'QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN/added_tokens.json',
 'QUANTIZED-MAIN-8BIT-LMSYS-PRETRAIN/tokenizer.json')